In [2]:
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:
target_dir = 'preprocessed'
file_pat = '*.txt'

In [4]:
# enron1からenron6までの6つのディレクトリからデータを読み込む
text = []

# enron1からenron6までループ
for i in range(1, 7):
    enron_dir = os.path.join(target_dir, f'enron{i}')
    
    # hamとspamの2つのカテゴリに対して処理
    for category in ['ham', 'spam']:
        category_dir = os.path.join(enron_dir, category)
        full_pat = os.path.join(category_dir, file_pat)
        n_files = 0
        
        # 各テキストファイルを読み込む
        for file in glob.glob(full_pat):
            n_files += 1
            try:
                with open(file, 'r', encoding='utf-8', errors='ignore') as f:
                    body = f.read()
                    text.append([category, body])
            except Exception as e:
                print(f'Error reading {file}: {e}')
        
        print(f'{n_files} files read for enron{i}/{category}')

print(f'\nTotal emails: {len(text)}')

599 files read for enron1/ham
508 files read for enron1/spam
505 files read for enron2/ham
490 files read for enron2/spam
536 files read for enron3/ham
506 files read for enron3/spam
663 files read for enron4/ham
645 files read for enron4/spam
453 files read for enron5/ham
779 files read for enron5/spam
582 files read for enron6/ham
710 files read for enron6/spam

Total emails: 6976


In [5]:
df = pd.DataFrame(text, columns=['category', 'text'])
print(f'データセットの形状: {df.shape}')
print(f'\nhamとspamの数:')
print(df['category'].value_counts())
df.head()

データセットの形状: (6976, 2)

hamとspamの数:
category
spam    3638
ham     3338
Name: count, dtype: int64


,category,text
0,ham,Subject: christmas tree farm pictures\n
1,ham,"Subject: vastar resources , inc .\ngary , prod..."
2,ham,Subject: calpine daily gas nomination\n- calpi...
3,ham,Subject: re : issue\nfyi - see note below - al...
4,ham,Subject: meter 7268 nov allocation\nfyi .\n- -...


In [6]:
X = df['text']
y = df['category']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

print(f'訓練データの数: {len(X_train)}')
print(f'テストデータの数: {len(X_test)}')

訓練データの数: 5580
テストデータの数: 1396


In [8]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

# 語彙(単語数)の取得
vocab = vectorizer.get_feature_names_out()
print(f'訓練データの中に含まれる単語数: {len(vocab)}')

# BoWの作成
X_train_bow = vectorizer.transform(X_train)
X_test_bow = vectorizer.transform(X_test)

訓練データの中に含まれる単語数: 65892


#### 単純ベイズ分類器の学習

In [9]:
model = MultinomialNB(alpha=1.0)
model.fit(X_train_bow, y_train)
print(f'分類クラス: {model.classes_}')

分類クラス: ['ham' 'spam']


#### 訓練データに対する予測精度

In [10]:
train_score = model.score(X_train_bow, y_train)
print(f'訓練データに対する予測精度: {train_score:.3f}')

訓練データに対する予測精度: 0.994


#### テストデータに対する予測精度

In [11]:
test_score = model.score(X_test_bow, y_test)
print(f'テストデータに対する予測精度: {test_score:.3f}')

テストデータに対する予測精度: 0.981


#### 結果のまとめ

In [12]:
print('=' * 50)
print('【結果のまとめ】')
print('=' * 50)
print(f'訓練データの数: {len(X_train)}')
print(f'テストデータの数: {len(X_test)}')
print(f'訓練データの中に含まれる単語数: {len(vocab)}')
print(f'訓練データに対する予測精度: {train_score:.3f}')
print(f'テストデータに対する予測精度: {test_score:.3f}')
print('=' * 50)

【結果のまとめ】
訓練データの数: 5580
テストデータの数: 1396
訓練データの中に含まれる単語数: 65892
訓練データに対する予測精度: 0.994
テストデータに対する予測精度: 0.981
